In [1]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 59.7 MB/s eta 0:00:00


In [2]:
import zipfile
import os

# Path to the ZIP file in Colab (for example, from your Google Drive)
zip_file_path = '/content/sample_data/zolvit.zip'  # Replace with your actual ZIP file path

# Directory where you want to extract the contents
extract_to_dir = '/content/extracted_folder/'  # Replace with your desired extraction path

# Create the directory if it doesn't exist
os.makedirs(extract_to_dir, exist_ok=True)

# Extract the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_dir)

print(f"Files extracted to: {extract_to_dir}")


Files extracted to: /content/extracted_folder/


In [4]:
import os
import csv
import re
import pdfplumber
import pandas as pd
# Directory containing your PDFs
pdf_directory = '/content/extracted_folder'
output_csv = 'invoice_data_2.csv'

def extract_table_from_pdf(pdf_path, page_num=0):
    # Define bounding boxes for each column based on x-coordinates
    columns = {
        "Item": (50, 250),           # Item description column
    }

    # Open the PDF and access the specified page
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num]

        # Create empty lists to hold column data
        data = {col: [] for col in columns}
        start_extracting = False  # Flag to start extracting data after "Item" header

        # Find the position of "Taxable Amount" to determine y_end
        y_start = 0
        y_end = None  # Initialize y_end as None

        # Read all lines to find "Taxable Amount" position
        lines = page.extract_text().splitlines()
        for line in lines:
            if "Taxable Amount" in line:
                # Get the y-coordinate of the line
                # Using the line number as a rough estimate for y-coordinate

                y_end = lines.index(line) * 20   # Assuming approx. 15 pixels per line, adjust as necessary
                break  # Stop searching after finding "Taxable Amount"

        if y_end is None:
            y_end = 600  # Fallback if "Taxable Amount" is not found

        for col_name, (x_start, x_end_col) in columns.items():
            # Crop each column area from the page within the specified bounding box
            cropped = page.within_bbox((x_start, y_start, x_end_col, y_end))
            text = cropped.extract_text()

            # Process text if available
            if text:
                processed_text = []
                for line in text.splitlines():
                    # Check if we should start extracting data
                    if "Item" in line:
                        start_extracting = True  # Set flag to true when we detect "Item" header
                        continue  # Skip the header line

                    if start_extracting:
                        line = line.strip()
                                          # Check for "ems" or "Qty" to stop extracting further text
                        if 'ems' in line.lower() or 'qty' in line.lower():
                            stop_extracting = True  # Set the flag to stop extracting
                            break  # Exit the loop if we find "ems" or "Qty"
                        # Handle hyphenated lines by checking if the current line ends with a hyphen
                        # print(processed_text)
                        if(line.startswith('&')):


                            line = re.sub(r'\b(?<!\d)(\d+)(?!([a-zA-Z]|\'\w*))\b', '', line)
                            processed_text[-1]  += (' '+line)

                        elif("Face Wash Gel" in line):

                            line = re.sub(r'\b(?<!\d)(\d+)(?!([a-zA-Z]|\'\w*))\b', '', line)
                            processed_text[-1]  += (' '+line)


                        # elif (processed_text and processed_text[-1].endswith(' - |- |-| \d+|&|& ')):
                        elif (processed_text and (processed_text[-1].endswith(' - ') or processed_text[-1].endswith('- ') or processed_text[-1].endswith('-') or processed_text[-1].endswith(' \d+') or processed_text[-1].endswith('& ') or processed_text[-1].endswith('& '))):
                          # Remove standalone integers not followed by letters (but keep decimals)
                            # line = re.sub(r'\b(?<!\d)\d+\b(?!\s*[a-zA-Z])', '', line)

                            line = re.sub(r'\b(?<!\d)(\d+)(?!([a-zA-Z]|\'\w*))\b', '', line)

                            processed_text[-1] += (' '+line)   # Remove hyphen and join with previous line

                        else:

                            line = line.replace(';', '\n')

                            # Keep valid decimal numbers with units intact
                            # line = re.sub(r'(?<!\d)(\d+(\.\d+)?)\s*(mg|ml|ML|MG)\b', r'\1 \3', line)  # Correctly match numbers with units
                            # line = re.sub(r'(?<!\d)(\d+(\.\d+)?|\d+)\s*(mg|ml|ML|MG)\b', r'\1 \3', line)  # Match both integers and decimals with units
                            line = re.sub(r'(\d+(\.\d+)?|\d+)\s*(mg|ml|ML|MG)', r'\1 \3', line)

                            line = re.sub(r'(?<!\d)(-\d+(\.\d+)?)\s*\b', '', line)  # Remove negative numbers
                            line = re.sub(r'\b\d+\.\d{2}\b', '', line)  # Remove numbers with exactly two decimal places

                            # Remove standalone integers not followed by letters (but keep decimals)
                            # line = re.sub(r'\b(?<!\d)\d+\b(?!\s*[a-zA-Z])', '', line)
                            line = re.sub(r'\b(?<!\d)(\d+)(?!(\s*[a-zA-Z]|\'\w*))\b', '', line)


                            line = re.sub(r'(?<!\d)\.(?!\d)', '', line)  # Remove standalone periods while preserving decimals within text

                            # Merge lines ending with a comma with the previous line
                            if processed_text and line.endswith(','):
                                processed_text[-1] += ' ' + line[:-1]  # Remove comma and merge
                            elif line.strip():  # Only append non-empty lines
                                processed_text.append(line)

                data[col_name] = processed_text

        # Adjust length of each column list to match the longest column
        max_len = max(len(values) for values in data.values())
        for key in data:
            data[key] += [''] * (max_len - len(data[key]))

    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

def extract_invoice_data(pdf_path):
    extracted_data = ""

    # Use pdfplumber to read PDF
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_data += page.extract_text() + "\n"  # Extract text from each page

    # Patterns for extracting invoice details
    patterns = {
        "invoice_number": r"Invoice #:\s*(INV-\d+)",  # Invoice number
        "invoice_date": r"Invoice Date:\s*(\d{2}\s[A-Za-z]{3}\s\d{4})",  # Invoice date
        "due_date": r"Due Date:\s*(\d{2}\s[A-Za-z]{3}\s\d{4})",  # Due date
        "taxable_amount": r"Taxable Amount ₹([\d,.]+)",  # Taxable amount
        "cgst": r"CGST\s+\d+\.\d+\%\s+₹([\d,.]+)",  # CGST amount
        "sgst": r"SGST\s+\d+\.\d+\%\s+₹([\d,.]+)",  # SGST amount
        "total_amount": r"Total ₹([\d,.]+)",  # Total amount
        "total_discount": r"Total Discount ₹([\d,.]+)",  # Total discount
        "total_items_qty": r"Total Items / Qty\s*:\s*(\d+)\s*/\s*(\d+)",  # Total items / qty
        "amount_in_words": r"Total amount \(in words\):\s*(.*)",  # Amount in words

    }

    invoice_data = {}

    # Extract key fields using the defined patterns
    for key, pattern in patterns.items():
        match = re.search(pattern, extracted_data, re.MULTILINE)
        if match:
            invoice_data[key] = match.group(1).strip()
        else:
            # Set default values for CGST and SGST to 0 if not found
            if key in ["cgst", "sgst"]:
                invoice_data[key] = 0
            else:
                invoice_data[key] = None


    # Regex pattern for customer details
    customer_details_pattern = r"Customer Details:\s*(.*?)\n"

    # Apply the pattern to the text
    match = re.search(customer_details_pattern, extracted_data, re.DOTALL)

    if match:
        customer_details_line = match.group(1).strip()  # Capture the customer details line

        # Check if the captured customer name is "Shipping Address:"
               # Check if the captured customer details are "Shipping Address:" or "Billing Address:"
        if customer_details_line == "Shipping Address:" or customer_details_line == "Billing Address:":
            # Create a regex pattern to match either "Shipping Address:" or "Billing Address:"
            address_pattern = r"(Shipping Address:|Billing Address:)\s*(\S+\s+\S+)"
            address_match = re.search(address_pattern, extracted_data)
            if address_match:
                customer_name = address_match.group(2)  # Get the next two words after either address
            else:
                customer_name = ""  # Default to empty if not found
        else:
            customer_name = customer_details_line  # Otherwise, use the captured name

        invoice_data["customer_name"] = customer_name
        invoice_data["shipping_address"] = ""  # Assuming you don't need to extract shipping address


    # Extract the item column from the PDF
    item_df = extract_table_from_pdf(pdf_path)
    invoice_data["items"] = item_df["Item"].tolist() if not item_df.empty else []




    return invoice_data

# Function to process all PDF files in the directory
def process_all_pdfs(pdf_directory):
    extracted_data_list = []

    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            print(f"Processing {filename}...")

            # Extract invoice data from the PDF
            invoice_data = extract_invoice_data(pdf_path)

            # Extract invoice number from the filename (if not found in text)
            if not invoice_data.get("invoice_number"):
                invoice_data["invoice_number"] = extract_invoice_number_from_filename(filename)

            extracted_data_list.append(invoice_data)

    return extracted_data_list

# Function to save extracted data to a CSV file
def save_data_to_csv(extracted_data_list, output_csv):
    with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([  # CSV header
            "Invoice Number", "Invoice Date", "Due Date", "Customer Name","Items",
             "Taxable Amount", "CGST", "SGST", "Total Amount",
            "Total Discount", "Total Items / Qty", "Amount in Words"
        ])

        for data in extracted_data_list:
            writer.writerow([
                data["invoice_number"],
                data["invoice_date"],
                data["due_date"],
                data["customer_name"],
                '\n'.join(data.get("items", [])),
                data["taxable_amount"],
                data["cgst"],
                data["sgst"],
                data["total_amount"],
                data["total_discount"],
                data["total_items_qty"],
                data["amount_in_words"],
            ])

def print_accuracy(extracted_data_list):
    total_invoices = len(extracted_data_list)
    column_names = [
        "invoice_number", "invoice_date", "due_date", "customer_name",
        "taxable_amount", "cgst", "sgst", "total_amount",
        "total_discount", "total_items_qty", "amount_in_words"
    ]

    print("\nAccuracy of each column:")
    for column in column_names:
        filled_count = sum(1 for data in extracted_data_list if data[column] is not None)
        accuracy = (filled_count / total_invoices) * 100 if total_invoices > 0 else 0
        print(f"{column.replace('_', ' ').title()}: {filled_count}/{total_invoices}  ({accuracy:.2f}%)")

    # Calculate accuracy for items based on lines vs total quantity
    # print("\nItem Count Accuracy:")
    c = 0
    error=[]
    for data in extracted_data_list:
        items = data.get("items", [])
        item_count = len(items)  # Number of lines in the items column

        # Extract total quantity safely
        total_quantity = 0
        total_items_qty = data.get("total_items_qty", "")
        total_quantity=int(total_items_qty)
        # print(total_items_qty)



        if total_quantity > 0:
            if(item_count == total_quantity):
                c=c+1
            else:
                inv = data.get("invoice_number", [])
                error.append(inv)

    item_accuracy = (c / total_invoices) * 100 if total_invoices > 0 else 0


    print(f"Item Accuracy:{c}/{total_invoices} ({item_accuracy:.2f}%)")
    for inv in error: print(f"Item Error in file number: {inv}")


# Main execution
if __name__ == "__main__":
    extracted_data = process_all_pdfs(pdf_directory)
    save_data_to_csv(extracted_data, output_csv)
    print_accuracy(extracted_data)

print(f"Data extraction complete. Check {output_csv} for results.")


Processing INV-135_Mohith Saragur.pdf...
Processing INV-129_Divya Suhane.pdf...
Processing INV-118_Rashu.pdf...
Processing INV-103_Jaiprakash Kumawat.pdf...
Processing INV-99_Indraja.pdf...
Processing INV-145_Indraja Mohite.pdf...
Processing INV-140_Ankit.pdf...
Processing INV-149_Karishma Bande.pdf...
Processing INV-112_Gauri.pdf...
Processing INV-144_Atia Latif.pdf...
Processing INV-133_Sheetal Kapur.pdf...
Processing INV-117_Naman.pdf...
Processing INV-142_Urmila Jangam.pdf...
Processing INV-116_Shefali.pdf...
Processing INV-104_Joseph Wincet.pdf...
Processing INV-150_Bhusan Naresh.pdf...
Processing INV-136_Rishabh Ramola.pdf...
Processing INV-106_Kamakshi Thakkar.pdf...
Processing INV-138_Agrani Kandele.pdf...
Processing INV-113_Raghvendra.pdf...
Processing INV-141_Kasturi Kalwar.pdf...
Processing INV-147_Divya Suhane.pdf...
Processing INV-73_Avik Mallick.pdf...
Processing INV-102_Kasturi Kalwar.pdf...
Processing INV-124_Ankita Sattva.pdf...
Processing INV-107_Prashant.pdf...
Proce